In [268]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

In [269]:
train = pd.read_csv(r'hr_temp_20170201-20200131_subset.csv', parse_dates = ['DATE'])
temp_eval = pd.read_csv(r'hr_temp_20200201-20200229_subset.csv', parse_dates = ['DATE'])
hrl = pd.read_csv(r'hrl_load_metered - 20170201-20200131.csv', parse_dates = ['datetime_beginning_utc','datetime_beginning_ept'])

In [270]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26280 entries, 0 to 26279
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   STATION                   26280 non-null  int64         
 1   DATE                      26280 non-null  datetime64[ns]
 2   REPORT_TYPE               26280 non-null  object        
 3   SOURCE                    26280 non-null  int64         
 4   HourlyDryBulbTemperature  26243 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 1.0+ MB


In [271]:
hrl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26280 entries, 0 to 26279
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   datetime_beginning_utc  26280 non-null  datetime64[ns]
 1   datetime_beginning_ept  26280 non-null  datetime64[ns]
 2   nerc_region             26280 non-null  object        
 3   mkt_region              26280 non-null  object        
 4   zone                    26280 non-null  object        
 5   load_area               26280 non-null  object        
 6   mw                      26280 non-null  float64       
 7   is_verified             26280 non-null  bool          
dtypes: bool(1), datetime64[ns](2), float64(1), object(4)
memory usage: 1.4+ MB


In [272]:
train['DATE'] = train['DATE'].dt.floor('h')

In [273]:
hrl['DATE'] = hrl['datetime_beginning_utc'] + timedelta(hours = -5)

In [274]:
hrl.head()

,datetime_beginning_utc,datetime_beginning_ept,nerc_region,mkt_region,zone,load_area,mw,is_verified,DATE
0,2017-02-01 05:00:00,2017-02-01 00:00:00,RFC,WEST,DUQ,DUQ,1419.881,True,2017-02-01 00:00:00
1,2017-02-01 06:00:00,2017-02-01 01:00:00,RFC,WEST,DUQ,DUQ,1379.505,True,2017-02-01 01:00:00
2,2017-02-01 07:00:00,2017-02-01 02:00:00,RFC,WEST,DUQ,DUQ,1366.106,True,2017-02-01 02:00:00
3,2017-02-01 08:00:00,2017-02-01 03:00:00,RFC,WEST,DUQ,DUQ,1364.453,True,2017-02-01 03:00:00
4,2017-02-01 09:00:00,2017-02-01 04:00:00,RFC,WEST,DUQ,DUQ,1391.265,True,2017-02-01 04:00:00


In [275]:
train.head()

,STATION,DATE,REPORT_TYPE,SOURCE,HourlyDryBulbTemperature
0,72520514762,2017-02-01 00:00:00,FM-15,7,37.0
1,72520514762,2017-02-01 01:00:00,FM-15,7,37.0
2,72520514762,2017-02-01 02:00:00,FM-15,7,36.0
3,72520514762,2017-02-01 03:00:00,FM-15,7,36.0
4,72520514762,2017-02-01 04:00:00,FM-15,7,36.0


In [276]:
train = pd.merge(train, hrl, how = 'left')

In [277]:
drop_cols = train.columns.tolist()
drop_cols.remove('mw')
drop_cols.remove('HourlyDryBulbTemperature')
drop_cols.remove('DATE')
train = train.drop(columns = drop_cols)
train.head()

,DATE,HourlyDryBulbTemperature,mw
0,2017-02-01 00:00:00,37.0,1419.881
1,2017-02-01 01:00:00,37.0,1379.505
2,2017-02-01 02:00:00,36.0,1366.106
3,2017-02-01 03:00:00,36.0,1364.453
4,2017-02-01 04:00:00,36.0,1391.265


In [278]:
train['hour'] = train['DATE'].dt.hour
train['weekday'] = train['DATE'].dt.dayofweek
train['month'] = train['DATE'].dt.month
train['year'] = train['DATE'].dt.year
train.columns = train.columns.str.lower()
train = train.set_index('date')

In [279]:
train = train.rename(columns = {'hourlydrybulbtemperature':'temp'})
train = train.reindex(columns = ['mw','temp','hour','weekday','month','year'])

In [280]:
train.head(30)

,mw,temp,hour,weekday,month,year
date,,,,,,
2017-02-01 00:00:00,1419.881,37.0,0,2,2,2017
2017-02-01 01:00:00,1379.505,37.0,1,2,2,2017
2017-02-01 02:00:00,1366.106,36.0,2,2,2,2017
2017-02-01 03:00:00,1364.453,36.0,3,2,2,2017
2017-02-01 04:00:00,1391.265,36.0,4,2,2,2017
2017-02-01 05:00:00,1477.343,36.0,5,2,2,2017
2017-02-01 06:00:00,1585.252,34.0,6,2,2,2017
2017-02-01 07:00:00,1671.412,33.0,7,2,2,2017
2017-02-01 08:00:00,1708.597,34.0,8,2,2,2017


In [281]:
train = train.interpolate()

In [282]:
train.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 26280 entries, 2017-02-01 00:00:00 to 2020-01-31 23:00:00
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   mw       26280 non-null  float64
 1   temp     26280 non-null  float64
 2   hour     26280 non-null  int64  
 3   weekday  26280 non-null  int64  
 4   month    26280 non-null  int64  
 5   year     26280 non-null  int64  
dtypes: float64(2), int64(4)
memory usage: 1.4 MB


In [283]:
test = train[train['year'] == 2020]
train = train[train['year'] < 2020]

In [284]:
train.tail()

,mw,temp,hour,weekday,month,year
date,,,,,,
2019-12-31 19:00:00,1606.134,32.0,19,1,12,2019
2019-12-31 20:00:00,1565.014,32.0,20,1,12,2019
2019-12-31 21:00:00,1515.905,32.0,21,1,12,2019
2019-12-31 22:00:00,1474.519,30.0,22,1,12,2019
2019-12-31 23:00:00,1421.717,30.0,23,1,12,2019


In [285]:
test.head()

,mw,temp,hour,weekday,month,year
date,,,,,,
2020-01-01 00:00:00,1363.428,31.0,0,2,1,2020
2020-01-01 01:00:00,1335.975,29.0,1,2,1,2020
2020-01-01 02:00:00,1296.817,30.0,2,2,1,2020
2020-01-01 03:00:00,1288.403,30.0,3,2,1,2020
2020-01-01 04:00:00,1292.263,31.0,4,2,1,2020
